In [ ]:
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

from pdflight import in_task, shift_datetime

%matplotlib inline

In [ ]:
with open('store.pkl', 'rb') as f:
    stored = pickle.load(f)
    md, fl, th = stored['md'], stored['fl'], stored['th']

In [ ]:
competition = md.index[md.competition == "23rd-fai-european-gliding-championships-bekescsaba-2025"]
if len(competition):
    md = md.loc[competition]
    fl = fl.loc[competition]
    th = th.loc[competition]

In [ ]:
tt = th.loc[in_task(th, md) & (th.dt.total_seconds() >= 120)]
tt

In [ ]:
df = pd.DataFrame(index=tt.index)
exit = [(i[0], i[1] + tt.loc[i]) for i in tt.index]
stable = [(i[0], i[1] + pd.Timedelta(seconds=40)) for i in tt.index]
df["v"] = (fl.loc[exit].alt.values - fl.loc[tt.index].alt.values) / tt.dt.total_seconds().values
df["dn"] = (fl.loc[stable].alt.values - fl.loc[tt.index].alt.values) / 40
df["e"] = df.dn / df.v

In [ ]:
df

In [ ]:
cc = md[["pilot", "points"]].join(df.groupby("flight").mean(), how="inner")

In [ ]:
sns.regplot(data=cc, x='v', y='points', line_kws={"color": "red"})
plt.show()

In [ ]:
res = stats.linregress(cc.v, cc.points)
res